In [ ]:
# import all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.decomposition import PCA
import prince
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import random
import csv